In [1]:
%matplotlib notebook
import IPython.display
from utils_demo import *
from sys import platform
import sys
from PIL import Image
import matplotlib.pyplot as plt
import json
import rdflib
import glob
import os
import re
import copy
import time

sys.path.append('../simulation')
sys.path.append('../dataset_utils/')

import numpy as np
import random
import cv2
import add_preconds
import augmentation_utils
import evolving_graph.check_programs as check_programs
import evolving_graph.utils as utils

from unity_simulator.comm_unity import UnityCommunication


Bad key "nbagg.transparent" on line 426 in
/Users/s-egami/.pyenv/versions/anaconda3-2020.07/lib/python3.8/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.2.2/matplotlibrc.template
or from the matplotlib source distribution

Bad key "animation.mencoder_path" on line 509 in
/Users/s-egami/.pyenv/versions/anaconda3-2020.07/lib/python3.8/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.2.2/matplotlibrc.template
or from the matplotlib source distribution

Bad key "animation.mencoder_args" on line 512 in
/Users/s-egami/.pyenv/versions/anaconda3-2020.07/lib/python3.8/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.2.2/

In [2]:
comm = UnityCommunication()

Select scene number. 
Please see https://github.com/xavierpuigf/virtualhome/tree/master/simulation#environment .

In [3]:
def generate_list_of_steps(file_path):
    file = open(file_path, "r", encoding="utf-8")
    i = 0
    list_of_steps = []
    program_name = ""
    description = ""
    char= "<char0>"
    while True:
        line = file.readline()
        if line:
            line = line.replace("\n","")
            if i==0:
                program_name = line
            elif i==1:
                description = line
            elif line.startswith("["):
                list_of_steps.append(line)
            else:
                pass
            i+=1
        else:
            break
    return program_name, description, list_of_steps

In [20]:
scene = 1
data_path = "../dataset/experiment_programs_202203/scene" + str(scene) + "/*.txt"
program_list = []
for file_path in sorted(glob.glob(data_path)):
    file_name = file_path.replace("../dataset/experiment_programs_202203/scene" + str(scene) + "/", "")
#     txt_file_name = file_name.split("/")[1]
    
    program_name, description, list_of_steps = generate_list_of_steps(file_path)
    program_list.append({
        "file_name":file_name,
        "name": program_name,
        "description": description,
        "list_of_steps": list_of_steps
    })

In [21]:
program_list

[{'file_name': 'Admire_paintings.txt',
  'name': 'Admire paintings',
  'description': 'Walk to paintings and admire them.',
  'list_of_steps': ['[WALK] <wallpictureframe> (419)',
   '[TURNTO] <wallpictureframe> (419)',
   '[LOOKAT] <wallpictureframe> (419)']},
 {'file_name': 'Browse_Internet.txt',
  'name': 'Browse internet',
  'description': 'I go to the office and go to the computer. I turn the computer on and use the internet.',
  'list_of_steps': ['[WALK] <livingroom> (336)',
   '[WALK] <computer> (434)',
   '[SWITCHON] <computer> (434)',
   '[WALK] <chair> (373)',
   '[SIT] <chair> (373)']},
 {'file_name': 'Brush_teeth.txt',
  'name': 'Brush teeth',
  'description': 'Walk to the bathroom and then walk to the sink. Grab the toothbrush. Switch on the faucet.',
  'list_of_steps': ['[WALK] <bathroom> (11)',
   '[WALK] <sink> (248)',
   '[GRAB] <toothbrush> (66)',
   '[SWITCHON] <faucet> (51)']},
 {'file_name': 'Carry_box.txt',
  'name': 'Carry box',
  'description': 'Walk to the box. 

In [6]:
def update_bbox(pre_graph, current_graph):
    try:
        new_graph= copy.deepcopy(current_graph)
        
        for pre_node in pre_graph["nodes"]:
            for new_node in new_graph["nodes"]:
                if new_node["id"] == pre_node["id"]:
                    new_node["bounding_box"] = pre_node["bounding_box"]
           
    except Exception as e:
        print(e)
    
    return new_graph

In [7]:
def update_states(pre_graph, graph_state, object_id_map):
    try:
        new_graph= copy.deepcopy(pre_graph)
        
        skip = [object_id_map[x] for x in object_id_map]
        for new_node in new_graph["nodes"]:
            new_node_id = new_node["id"]
            if new_node_id in skip:
                continue
            temp_node_id = copy.deepcopy(new_node_id)
            # scriptで使用したobject_idとgraph_stateで反映されたidが異なる場合
            if str(new_node_id) in object_id_map:
                temp_node_id = object_id_map[str(new_node_id)]
            for graph_state_node in graph_state["nodes"]:
                if temp_node_id == graph_state_node["id"]:
                    new_node["states"] = graph_state_node["states"]
                    new_node["properties"] = graph_state_node["states"]
           
    except Exception as e:
        print(e)
    
    return new_graph

In [8]:
def create_object_id_map(original_script, executed_program, g):
    try:
        object_id_map = {}
        object_name_map = {}
        for original_block_str, executed_block_str in zip(original_script, executed_program):
            original_action, original_obj_names_corr, original_inst_nums_corr = augmentation_utils.parseStrBlock(original_block_str)
            executed_action, executed_obj_names_corr, executed_inst_nums_corr = augmentation_utils.parseStrBlock(executed_block_str)
            
            #object_idのmapを作る
            for original_id, executed_id in zip(original_inst_nums_corr, executed_inst_nums_corr):
                if original_id not in object_id_map:
                    if original_id != executed_id:
                        object_id_map[original_id] = int(executed_id)
            
            #object_nameのmap作る
            for original_obj_name, original_id in zip(original_obj_names_corr, original_inst_nums_corr):
                g_objects = [x for x in g["nodes"] if x["id"] == original_id]
                if len(g_objects) != 0:
                    g_obj = g_objects[0]
                    if original_obj_name != g_obj["class_name"]:
                        object_name_map[original_obj_name] = g_obj["class_name"]
    except Exception as e:
        print(e)
    return object_id_map, object_name_map

In [9]:
#ランダムでid2000以降のobjectを追加してしまうので、削除する
def delete2000(graph_state_list):
    new_graph_state_list = []
    for graph_state in graph_state_list:
        new_nodes = [x for x in graph_state["nodes"] if x["id"] < 2000]
        new_edges = [x for x in graph_state["edges"] if x["from_id"] < 2000 and x["to_id"] < 2000]
        new_graph_state_list.append({"nodes": new_nodes, "edges": new_edges})
    return new_graph_state_list

In [12]:
def switchOff(g, class_names):
    for class_name in class_names:
        objects = find_nodes(g, class_name=class_name)
        for obj in objects:
            states = [x for x in obj["states"] if x != "ON"]
            if "OFF" not in states:
                states.append("OFF")
            obj["states"] = states
#             for node in g["nodes"]:
#                 if node["id"] == obj["id"]:
#                     node = obj
        return g

In [13]:
def export(activity_name, graph_state_list, time_list, script, object_name_map, out_dir_name):
    state_cnt = 0
    for graph_state in graph_state_list:
        state_cnt += 1
        file_path = "graph_state_list_with_bbox_png/" + out_dir_name + "_scene" + str(scene) + "/"  + activity_name + "/activityList-graph-state-" + '{0:03d}'.format(state_cnt) + ".json"
        with open(file_path, 'w') as outfile:
            json.dump(graph_state, outfile)

    with open("graph_state_list_with_bbox_png/" + out_dir_name + "_scene" + str(scene) + "/" + activity_name + "/activityList-program.txt", 'w') as f:
        for s in script:
            #ここでscriptのobject_nameを置換する処理書く
            for x in object_name_map:
                if x in s:
                    s = s.replace("<" + x  + ">", "<" + object_name_map[x] + ">")
            f.write("%s\n" % s)

    with open("graph_state_list_with_bbox_png/" + out_dir_name + "_scene" + str(scene) + "/" + activity_name + "/program-description.txt", 'w') as f:
        f.write("%s\n" % activity["name"])
        f.write("%s\n" % activity["description"])
        
    time_list = [str(time) for time in time_list]
    duration = "\n".join(time_list)
    with open("graph_state_list_with_bbox_png/" + out_dir_name + "_scene" + str(scene) + "/" + activity_name + "/duration.txt", 'w') as f:
        f.write(duration)

In [14]:
def check_script_type(script):
    find_solution = False
    fridge = False
    if "(1)" in script[0]:
        find_solution = True
    for x in script:
        if "[OPEN] <fridge>" in x:
            fridge = True
    return find_solution, fridge

In [15]:
def getRoom(g, script):
    result = None
    try:
        rooms = {"bedroom": 74, "livingroom": 336, "kitchen": 207, "bathroom": 11}
        action, obj_names_corr, inst_nums_corr = augmentation_utils.parseStrBlock(script[0])
        obj_name = obj_names_corr[-1]
        obj_id = inst_nums_corr[-1]
        if obj_name in rooms:
            result = obj_name
        else:
            edges = g["edges"]
            to_ids = [edge["to_id"] for edge in edges if edge["relation_type"] == "INSIDE" and edge["from_id"] == int(obj_id)]
            for to_id in to_ids:
                result = [k for k, v in rooms.items() if v == to_id][0]
    except Exception as e:
        print(e)
    return result

In [17]:
#find_solution = false
out_dir_name = "experiment_programs_202204"
for activity in program_list:
    activity_name = activity["name"]
    try:
        new_graph_state_list = []
        time_list = []
        comm.reset(scene-1)
        success, g = comm.environment_graph();
#         comm.add_character('chars/Female4')
        script = activity["list_of_steps"]
        # scriptの1行目を見て部屋を判定したい
        room = getRoom(g, script)
        comm.add_character(initial_room=room)
        success, g = comm.environment_graph();
        print(activity["file_name"])
        
        
        find_solution, fridge = check_script_type(script)
        script_for_check = []
        if find_solution:
            continue
        else:
            for x in script:
                script_for_check.append(re.sub("(\d+)","1",x))
            print("script for check")
            print(script_for_check)

        preconds = add_preconds.get_preconds_script(script).printCondsJSON()
        print(preconds)
        print("switch off")
        g = switchOff(g, ["computer", "tv", "faucet", "television"])
        
        g = check_programs.translate_graph_dict_nofile(g)
        info = check_programs.check_script(script, preconds, graph_path=None, inp_graph_dict=g)
        message, final_state, graph_state_list, graph_dict, id_mapping, info, helper, executed_script = info
        # if delete randomly placced objects
        graph_state_list = delete2000(graph_state_list)
        print(message)
        executed_program = []
        for x in executed_script:
            executed_program.append("<char0> " + re.sub("\[\d+\]","",x.__str__()).strip())
        print(executed_program)

        if message != "Script is executable":
            continue
        
        new_script = []
        
        for x in script:
            new_script.append("<char0> " + x)
            
        i = 0

        object_id_map, object_name_map = create_object_id_map(new_script, executed_program, g)
        print("object_id_map")
        print(object_id_map)
    
        success, g = comm.environment_graph()
#         g = update_bbox(g, graph_state_list[i])
        g = update_states(g, graph_state_list[i], object_id_map)
        new_graph_state_list.append(copy.deepcopy(g))
        
        for instruction in new_script:
            i += 1
            start = time.time()
            success, message = comm.render_script(script=[instruction],
                                      #processing_time_limit=80,
                                      find_solution=False,
                                      image_width=480,
                                      image_height=360,  
                                      #skip_animation=False,
                                      recording=True,
    #                                               gen_vid=True,
                                      #save_scene_states=True,
                                      file_name_prefix=activity_name,
                                      output_folder="Output/" + out_dir_name + "/scene" + str(scene),
                                      camera_mode=["PERSON_FROM_BACK", "AUTO"],
                                      frame_rate=30
                                     )
            time_list.append(time.time() - start)
            success, g = comm.environment_graph()
            print(str(success) + ": " + str(instruction))
#             g = update_bbox(g, graph_state_list[i])
            g = update_states(g, graph_state_list[i], object_id_map)
            new_graph_state_list.append(copy.deepcopy(g))
            
        if os.path.exists("graph_state_list_with_bbox_png/" + out_dir_name + "_scene" + str(scene) + "/" + activity_name) == False:
            os.mkdir("graph_state_list_with_bbox_png/" + out_dir_name + "_scene" + str(scene) + "/" + activity_name)
        export(activity_name, new_graph_state_list, time_list, new_script, object_name_map,  out_dir_name)
#                 with open("Output/" + activity_name + activity_cnt + "/" + str(scene) + "/ftaa_" + activity_name + activity_cnt + ".txt")  as f:
#                     line = f.readline()
#                     line.split(" ")
    except Exception as e:
        print(e)

Go_to_sleep.txt
script for check
['[WALK] <bedroom> (1)', '[WALK] <light> (1)', '[SWITCHOFF] <light> (1)', '[WALK] <bed> (1)', '[SIT] <bed> (1)']
[{'location': [['light', '175'], ['bedroom', '74']]}, {'location': [['bed', '111'], ['bedroom', '74']]}, {'free': ['bed', '111']}]
switch off
Script is executable
['<char0> [WALK] <bedroom> (74)', '<char0> [WALK] <light> (175)', '<char0> [SWITCHOFF] <light> (175)', '<char0> [WALK] <bed> (111)', '<char0> [SIT] <bed> (111)']
object_id_map
{}
True: <char0> [WALK] <bedroom> (74)
True: <char0> [WALK] <light> (175)
True: <char0> [SWITCHOFF] <light> (175)
True: <char0> [WALK] <bed> (111)
True: <char0> [SIT] <bed> (111)


# 以下、ffmpegで動画生成

In [18]:
#complementing missing images
try:
    for x in range(2):
        for activity in program_list:
            activity_name = activity["name"]
            img_path = "../simulation/Output/experiment_programs_202204/scene" + str(scene) + "/" + activity_name + "/0/??????_????_" + str(x) + "_normal.png"
            i = 0
            for file_path in sorted(glob.glob(img_path)):
                file_name = file_path.replace("../simulation/Output/experiment_programs_202204/scene" + str(scene) + "/" + activity_name + "/0/", "")
                file_num = file_name.split("_")[1]
                if i != int(file_num):
                    print(str(i) + " " + str(int(file_num)))
                    img = PIL.Image.open(file_path)
                    new_file_name = "../simulation/Output/experiment_programs_202204/scene" + str(scene) + "/" + activity_name + "/0/" + "Action_%04d_" + str(x) + "_normal.png"
                    img.save(new_file_name % i)
                    print(file_name)
                    print(file_name + " - 1")
                    print("save")
                    i += 1
                i += 1
except Exception as e:
    print(e)

54 55
Action_0055_0_normal.png
Action_0055_0_normal.png - 1
save
127 128
Action_0128_0_normal.png
Action_0128_0_normal.png - 1
save
177 178
Action_0178_0_normal.png
Action_0178_0_normal.png - 1
save
314 315
Action_0315_0_normal.png
Action_0315_0_normal.png - 1
save
54 55
Action_0055_1_normal.png
Action_0055_1_normal.png - 1
save
127 128
Action_0128_1_normal.png
Action_0128_1_normal.png - 1
save
177 178
Action_0178_1_normal.png
Action_0178_1_normal.png - 1
save
314 315
Action_0315_1_normal.png
Action_0315_1_normal.png - 1
save


In [19]:
import subprocess

dir_path = "../simulation/Output/experiment_programs_202204/scene1/"
try:
    for x in range(2):
        for activity in program_list:
            activity_name = activity["name"]
            sh = subprocess.check_output(["ffmpeg", "-framerate", "14.5", "-i", dir_path + activity_name + "/0/Action_%04d_" + str(x) + "_normal.png", "-vcodec", "libx264", "-pix_fmt", "yuv420p", "-r", "30", dir_path + activity_name + "/0/" + activity_name.replace(" ", "_") + str(x) + ".mp4"])
            print(sh)
except Exception as e:
    print(e)

b''
b''


In [33]:
for activity in program_list:
    activity_name = activity["name"]
    file_path = "../simulation/Output/experiment_programs_202204/scene1/" + activity_name + "/0/ftaa_" + activity_name + ".txt"
    with open(file_path)  as f:
        lines = f.readlines()
#         arr = line.split(" ")
        print(activity_name)
        print(lines)

Admire paintings
['0 WALK 0 72\n', '0 TURNTO 73 73\n', '0 TURNTO 74 73\n']
Browse internet
['0 WALK 0 93\n', '0 WALK 94 165\n', '0 WALK 166 212\n', '0 SWITCHON 213 262\n', '0 WALK 263 354\n', '0 WALK 355 402\n', '0 SIT 403 475\n']
Brush teeth
['0 WALK 0 36\n', '0 WALK 37 96\n', '0 GRAB 97 142\n', '0 WALK 143 164\n', '0 SWITCHON 165 213\n']
Carry box
['0 WALK 0 55\n', '0 GRAB 56 101\n', '0 WALK 102 387\n', '0 TURNTO 388 425\n', '0 PUTBACK 426 469\n']
Clean desk
['0 WALK 0 71\n', '0 GRAB 72 117\n', '0 WALK 118 281\n', '0 PUTBACK 282 325\n']
Drink
['0 WALK 0 153\n', '0 WALK 154 201\n', '0 GRAB 202 247\n', '0 WALK 248 326\n', '0 WALK 327 326\n', '0 SWITCHON 327 376\n', '0 WALK 377 427\n', '0 PUTBACK 428 472\n', '0 WALK 473 513\n', '0 SWITCHOFF 514 563\n', '0 GRAB 564 609\n', '0 DRINK 610 763\n']
Go to sleep
['0 WALK 0 55\n', '0 WALK 56 128\n', '0 WALK 129 128\n', '0 SWITCHOFF 129 178\n', '0 WALK 179 315\n', '0 WALK 316 365\n', '0 SIT 366 444\n']
Go to toilet
['0 WALK 0 43\n', '0 WALK 44 12

# 以下、使わない

In [10]:
def loadFoodInFridge(g):
    freezer = find_nodes(g, class_name="fridge")[0]
    freezer["states"] = ["OPEN"]
    add_node(g, {'class_name': 'pie', 
                   'category': 'Food', 
                   'id': 1000, 
                   'properties': [], 
                   'states': []})
    add_edge(g, 1000, 'INSIDE', freezer['id'])
    comm.expand_scene(g)
    success, g = comm.environment_graph()
    freezer = find_nodes(g, class_name="fridge")[0]
    freezer["states"] = ["CLOSED"]
    comm.expand_scene(g)
    success, g = comm.environment_graph()
    return success, g

In [11]:
def loadObjectInCabinet(g):
    try:
        cabinets = find_nodes(g, class_name="cabinet")
        cabinet = [x for x in cabinets if x["id"] == 416][0]

        cabinet["states"] = ["OPEN"]
        add_node(g, {'class_name': 'pie', 
                   'category': 'Food', 
                   'id': 1000, 
                   'properties': [], 
                   'states': []})
        add_edge(g, 1000, 'INSIDE', cabinet['id'])
        comm.expand_scene(g)
        success, g = comm.environment_graph()
        cabinets = find_nodes(g, class_name="cabinet")
        cabinet = [x for x in cabinets if x["id"] == 416][0]
        cabinet["states"] = ["CLOSED"]
        comm.expand_scene(g)
        success, g = comm.environment_graph()
    except Exception as e:
        print(e)
    return success, g

In [34]:
script = [
    "[WALK] <kitchen> (1)",
    "[WALK] <fridge> (1)",
    "[OPEN] <fridge> (1)",
    "[WALK] <pie> (1)",
    "[GRAB] <pie> (1)",
    "[CLOSE] <fridge> (1)",
    "[WALK] <microwave> (1)",
    "[OPEN] <microwave> (1)",
    "[PUTIN] <pie> (1) <microwave> (1)",
    "[CLOSE] <microwave> (1)",
    "[SWITCHON] <microwave> (1)"
]

print('Starting scene...')


try:
    message = comm.reset(scene-1)
    comm.add_character("Chars/Female4")
    success, g = comm.environment_graph();
    
    
    preconds = add_preconds.get_preconds_script(script).printCondsJSON()
    print(preconds)
    g = check_programs.translate_graph_dict_nofile(g)
    info = check_programs.check_script(script, preconds, graph_path=None, inp_graph_dict=g)
    message, final_state, graph_state_list, graph_dict, id_mapping, info, helper, executed_script = info
    graph_state_list = delete2000(graph_state_list)
    executed_program = []
    for x in executed_script:
        executed_program.append("<char0> " + re.sub("\[\d+\]","",x.__str__()).strip())
    if message == "Script is executable":
        comm.expand_scene(graph_state_list[0])
        success, g = comm.environment_graph()
        
        # put the food in the freezer
        success, g = loadFoodInFridge(g)


        print(executed_program)
        print('Generating video...')
        new_script = []
        for x in script:
            new_script.append("<char0> " + x)
        success, message = comm.render_script(script=new_script, #cook some food　では　new_script
                                              #processing_time_limit=80,
                                              find_solution=True, # cook some food では True
                                              image_width=480,
                                              image_height=360,  
                                              #skip_animation=False,
                                              recording=True,
#                                               gen_vid=True,
                                              #save_scene_states=True,
                                              file_name_prefix="Cook_some_food",
                                              output_folder="Output/experiment_programs_202203/scene1",
                                              frame_rate=20
                                             )
        print(message)
#         imgs_final = get_scene_cameras(comm, [40])
    else:
        print(message)
except Exception as e:
    print(e)

Starting scene...
[{'free': ['chair', '1']}]
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


In [ ]:
freezer = find_nodes(g, class_name="fridge")[0]
freezer["states"] = ["CLOSED"]
comm.expand_scene(g)
imgs_final = get_scene_cameras(comm, [40])

In [ ]:
message = comm.reset(scene-1)
success, g = comm.environment_graph();
freezer = find_nodes(g, class_name="fridge")[0]
freezer["states"] = ["OPEN"]
comm.expand_scene(g)
imgs_final = get_scene_cameras(comm, [40])

In [ ]:
#電子レンジの上にグラス
message = comm.reset(scene-1)
success, g = comm.environment_graph();
sink = find_nodes(g, class_name="rug")[4]
add_node(g, {'class_name': 'pillow', 
               'category': 'placable_objects', 
               'id': 1000, 
               'properties': [], 
               'states': []})
add_edge(g, 1000, 'ON', sink['id'])
add_edge(g, 1000, 'CLOSE', 369)
comm.expand_scene(g)
success, g = comm.environment_graph()
imgs_final = get_scene_cameras(comm, [59])

In [210]:
message = comm.reset(0)
success, g = comm.environment_graph();
suceess, message = comm.get_visible_objects(57)
imgs_final = get_scene_cameras(comm, [57])
message

{'207': 'kitchen',
 '336': 'livingroom',
 '369': 'sofa',
 '372': 'coffeetable',
 '418': 'rug',
 '422': 'pillow',
 '423': 'pillow',
 '438': 'apple',
 '439': 'apple',
 '440': 'bananas',
 '442': 'peach',
 '443': 'peach',
 '444': 'plum',
 '445': 'plum',
 '446': 'dishbowl',
 '450': 'cellphone'}

In [338]:
message = comm.reset(scene-1)
comm.add_character()
success, g = comm.environment_graph();
script = [
    "[WALK] <tv> (427)",
    "[SWITCHON] <tv> (427)",
    "[WALK] <sofa> (369)",
    "[TURNTO] <tv> (427)",
    "[SIT] <sofa> (369)",
    "[WATCH] <tv> (427)"
#     "[DRINK] <wineglass> (199)",
#     "[WALK] <kitchen> (207)",
#     "[WALK] <sink> (247)",
#     "[PUTBACK] <wineglass> (199)"
#     "[WALK] <floor> (82)",
#     "[PUTBACK] <pillow> (423) <floor> (82)"
#     "[GRAB] <hanger> (115)",
#     "[WALK] <floor> (79)", 
#     "[TURNTO] <wallpictureframe> (419)",
#     "[LOOKAT] <wallpictureframe> (419)"
#     "[WALK] <rug> (179)",
#     "[PUTBACK] <hanger> (115) <floor> (79)",
#     "[WALK] <livingroom> (336)",
#     "[WALK] <sofa> (369)",
#     "[SIT] <sofa> (369)"
#     "[WALK] <computer> (1)",
#     "[SWITCHON] <computer> (1)",
#     "[WALK] <chair> (1)",
#     "[SIT] <chair> (1)"
#     "[OPEN] <washing_machine> (73)"
#     "[PUTBACK] <paper> (303) <desk> (110)"
#     "[GRAB] <clock> (266)",
#     "[WALK] <livingroom> (336)",
#     "[WALK] <coffeetable> (372)",
#     "[PUTBACK] <clock> (266) <coffeetable> (372)"
#     "[PUTBACK] <mug> (196) <sink> (247)"
#     "[SIT] <chair> (109)"
#     "[STANDUP]",
#     "[TURNTO] <toilet> (46)",
#     "[TOUCH] <toilet> (46)",
#     "[WALK] <sink> (248)",
#     "[SWITCHON] <faucet> (51)",
#     "[SWITCHOFF] <faucet> (51)"
#     "[GRAB] <plate> (274)",
#     "[WALK] <kitchencounter> (238)",
#     "[PUTBACK] <plate> (274) <kitchencounter> (238)",
#     "[WALK] <breadslice> (310)",
#     "[GRAB] <breadslice> (310)",
#     "[GRAB] <breadslice> (311)",
#     "[WALK] <plate> (274)",
#     "[PUTBACK] <breadslice> (310) <plate> (274)",
#     "[PUTBACK] <breadslice> (311) <plate> (274)",
#     "[WALK] <salmon> (328)",
#     "[GRAB] <salmon> (328)",
#     "[WALK] <plate> (274)",
#     "[WALK] <coffeepot> (308)",
#     "[GRAB] <coffeepot> (308)",
#     "[WALK] <kitchentable> (231)",
#     "[PUTBACK] <coffeepot> (308) <kitchentable> (231)"
#     "[SWITCHON] <faucet> (249)",
#     "[WALK] <sink> (247)",
#     "[PUTBACK] <glass> (271) <sink> (247)",
#     "[SWITCHOFF] <faucet> (249)",
#     "[GRAB] <glass> (271)",
#     "[DRINK] <glass> (271)"
]
try:
    preconds = add_preconds.get_preconds_script(script).printCondsJSON()
    print(preconds)
    g = check_programs.translate_graph_dict_nofile(g)
    info = check_programs.check_script(script, preconds, graph_path=None, inp_graph_dict=g)
    message, final_state, graph_state_list, graph_dict, id_mapping, info, helper, executed_script = info
    
#     comm.expand_scene(graph_state_list[0])
    success, g = comm.environment_graph()
    
    executed_program = []
    for x in executed_script:
        instruction = "<char0> " + re.sub("\[\d+\]","",x.__str__()).strip()
#         instruction = instruction.replace("cup", "waterglass")
        executed_program.append(instruction)
#     graph_state_list = delete2000(graph_state_list)
    print(message)
    print(executed_program)
    new_script = []
    for x in script:
        new_script.append("<char0> " + x)
    success, message = comm.render_script(script=new_script, #cook some food　では　new_script
                                                  #processing_time_limit=80,
                                                  find_solution=False, # cook some food では True
                                                  image_width=480,
                                                  image_height=360,  
                                                  #skip_animation=False,
                                                  recording=True,
    #                                               gen_vid=True,
                                                  #save_scene_states=True,
    #                                               file_name_prefix=activity_name,
                                                  output_folder="Output/test",
#                                                   camera_mode=[58],
                                                  frame_rate=20
                                                 )
except Exception as e:
    print(e)

[{'atreach': [['tv', '427'], ['sofa', '369']]}, {'free': ['sofa', '369']}]
Script is executable
['<char0> [WALK] <television> (427)', '<char0> [SWITCHON] <television> (427)', '<char0> [WALK] <couch> (369)', '<char0> [TURNTO] <television> (427)', '<char0> [SIT] <couch> (369)', '<char0> [WATCH] <television> (427)']


In [155]:
executed_program

['<char0> [WALK] <dresser> (1000)',
 '<char0> [GRAB] <hanger> (115)',
 '<char0> [WALK] <bedroom> (74)',
 '<char0> [PUTBACK] <hanger> (115) <bedroom> (74)']

In [ ]:
imgs_final = get_scene_cameras(comm, [42])

In [161]:
sink = find_nodes(g, class_name="floor")
sink

[{'id': 12,
  'category': 'Floor',
  'class_name': 'floor',
  'prefab_name': 'PRE_FLO_Tile_07',
  'obj_transform': {'position': [-6.385, -0.003, -0.527],
   'rotation': [0.0, 0.0, 0.0, 1.0],
   'scale': [1.0, 1.0, 1.0]},
  'bounding_box': {'center': [-7.635, -0.003, -0.52699995],
   'size': [2.5, 9.050127e-06, 2.5]},
  'properties': ['SURFACES'],
  'states': []},
 {'id': 13,
  'category': 'Floor',
  'class_name': 'floor',
  'prefab_name': 'PRE_FLO_Tile_07',
  'obj_transform': {'position': [-6.385, -0.003, 1.973],
   'rotation': [0.0, 0.0, 0.0, 1.0],
   'scale': [1.0, 1.0, 1.0]},
  'bounding_box': {'center': [-7.635, -0.003, 1.973],
   'size': [2.5, 9.050127e-06, 2.5]},
  'properties': ['SURFACES'],
  'states': []},
 {'id': 14,
  'category': 'Floor',
  'class_name': 'floor',
  'prefab_name': 'PRE_FLO_Tile_07',
  'obj_transform': {'position': [-3.88500023, -0.003, 1.973],
   'rotation': [0.0, 0.0, 0.0, 1.0],
   'scale': [1.0, 1.0, 1.0]},
  'bounding_box': {'center': [-5.135, -0.003, 1.97

In [ ]:
cabinets = find_nodes(g, class_name="cabinet")
cabinet = [x for x in cabinets if x["id"] == 416][0]

cabinet["states"] = ["OPEN"]
comm.expand_scene(g)
imgs_final = get_scene_cameras(comm, [61])

In [ ]:
objects = find_nodes(g, class_name="mousemat")
objects

In [ ]:
loadObjectInCabinet(g)

In [64]:
action

'WALK'

In [65]:
obj_names

['sink']

In [66]:
inst_nums

['247']